## 01/03/19
- Investigating the use of mrmr as feature selection method.

In [1]:
import sys
sys.path.insert(0, '/Users/matthewashman/github/MasterProject2018')

import numpy as np
import pandas as pd
from FeatureExtraction import mi_ksg 
from FeatureExtraction.npeet import entropy_estimators
from sklearn.feature_selection import mutual_info_classif

import pdb

In [2]:
cs12_features = pd.read_csv('/Users/matthewashman/github/MasterProject2018/FeatureSelection/cs12_features_for_mrmr.csv')
cs34_features = pd.read_csv('/Users/matthewashman/github/MasterProject2018/FeatureSelection/cs34_features_for_mrmr.csv')
cs56_features = pd.read_csv('/Users/matthewashman/github/MasterProject2018/FeatureSelection/cs56_features_for_mrmr.csv')

cs12_features = cs12_features.drop(['Unnamed: 0'], axis=1)
cs34_features = cs34_features.drop(['Unnamed: 0'], axis=1)
cs56_features = cs56_features.drop(['Unnamed: 0'], axis=1)

In [3]:
cs12_features.drop(['Label 1', 'Label 2'], axis=1).head()

,Approximate Entropy: m=3 r=0.7,Conduction Delay: set_thresh=False,Energy Ratio by Chunks: num_segments=10 segment_focus=2,Index Mass Quantile: q=0.6,Location of Maximum Energy: M=14,Maximum Absolute Value,Normalised Standard Deviation,Number of Peaks: set_thresh=False,Number of Peaks: set_thresh=True,Peaks Between Min and Max,Percentage Fractionation: thresh=0.01,Power Spectral Entropy,Power Spectral Entropy Around Maximum Energy,Ratio Beyond 5xSTD,Sample Entropy Around Max Energy: width=60 r=0.025,Spectral Centroid,Spectral Centroid Around Maximum Energy,Standard Deviation
0,-0.001151,7,0.106376,0.060000,6,-6356.0,0.033565,-1,0,0,-2.666667,-0.331105,1.001693,0.0,0.202478,-3.809282,2.096195,0.033827
1,-0.000091,6,0.064247,0.053333,5,-6345.0,0.034567,-1,0,0,-2.666667,-0.306431,1.137408,0.0,0.126912,-3.494683,2.321757,0.034745
2,-0.001086,7,0.092739,0.060000,6,-5604.0,0.027103,-1,0,0,-2.666667,-0.340625,0.707929,0.0,0.231244,-3.742084,1.777805,0.028154
3,-0.001582,7,0.076943,0.053333,5,-5613.0,0.032848,-1,0,0,-3.333333,-0.328423,1.209492,0.0,0.179761,-3.466419,2.462423,0.032945
4,-0.001070,7,0.104145,0.060000,6,-5932.0,0.031839,-1,0,0,-2.666667,-0.349868,1.105153,0.0,0.193480,-3.773304,2.280287,0.032160


In [4]:
X_cs12 = cs12_features.drop(['Label 1', 'Label 2', 'Power Spectral Entropy'], axis=1).values
feature_names = cs12_features.drop(['Label 1', 'Label 2', 'Power Spectral Entropy'], axis=1).columns
y_cs12 = cs12_features['Label 1'].values

In [24]:
def mrmr(X, y, K, feature_names):
    # Select the K best features using mrmr using greedy search
    num_features = X.shape[1]
    feature_idxs = np.arange(num_features)
    feature_y_mis = np.zeros(num_features)
    feature_y_mis = mutual_info_classif(X, y)
    
    # Best features. Begin with feature with the largest MI.
    features_to_keep = []
    features_to_keep.append(np.argmax(feature_y_mis))
    
    # Loop through all other features, and add the one that has the best relevancy-redundancy
    while(len(features_to_keep)<K):      
        # Features that are left
        features_left = [idx for idx in feature_idxs if (idx not in features_to_keep)]
        redundancies = np.zeros(len(features_left))
        for i, feature in enumerate(features_left):
            # Compute average redundancy of the feature
            for kept_feature in features_to_keep:
                mi = entropy_estimators.mi(X[:,kept_feature].reshape([X.shape[0],1]), X[:,feature].reshape([X.shape[0],1]))
                redundancies[i] += (1/(5*len(features_to_keep)+1))*max(0,mi)
        # Compute relevancy - redundancy
        score = feature_y_mis[features_left] - redundancies
        features_to_keep.append(features_left[np.argmax(score)])
            
    features_to_keep_mis = feature_y_mis[features_to_keep]
    feature_names_to_keep = feature_names[features_to_keep]
    
    return features_to_keep, features_to_keep_mis, feature_names_to_keep

In [25]:
ftk, ftk_mis, fntk = mrmr(X_cs12, y_cs12, 10, feature_names)

In [26]:
print(fntk)
print(feature_names)

Index(['Number of Peaks: set_thresh=False', 'Maximum Absolute Value',
       'Location of Maximum Energy: M=14',
       'Percentage Fractionation: thresh=0.01', 'Index Mass Quantile: q=0.6',
       'Ratio Beyond 5xSTD', 'Approximate Entropy: m=3 r=0.7',
       'Sample Entropy Around Max Energy: width=60 r=0.025',
       'Number of Peaks: set_thresh=True', 'Spectral Centroid'],
      dtype='object')
Index(['Approximate Entropy: m=3 r=0.7', 'Conduction Delay: set_thresh=False',
       'Energy Ratio by Chunks: num_segments=10 segment_focus=2',
       'Index Mass Quantile: q=0.6', 'Location of Maximum Energy: M=14',
       'Maximum Absolute Value', 'Normalised Standard Deviation',
       'Number of Peaks: set_thresh=False', 'Number of Peaks: set_thresh=True',
       'Peaks Between Min and Max', 'Percentage Fractionation: thresh=0.01',
       'Power Spectral Entropy Around Maximum Energy', 'Ratio Beyond 5xSTD',
       'Sample Entropy Around Max Energy: width=60 r=0.025',
       'Spectral Ce

In [ ]:
y.shape

In [49]:
x = [[1.3],[3.7],[5.1],[2.4],[3.4]]
y = [[1],[1],[0],[0],[0]]
entropy_estimators.micd(x,y)

TypeError: only integer scalar arrays can be converted to a scalar index

In [43]:
type(x[0])

list